In [22]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [25]:
MAX_LEN = 50
MAX_WORDS = 10000

In [12]:
! git clone https://github.com/CrowdTruth/Short-Text-Corpus-For-Humor-Detection.git

Cloning into 'Short-Text-Corpus-For-Humor-Detection'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 43 (delta 13), reused 43 (delta 13), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [13]:
%cd Short-Text-Corpus-For-Humor-Detection/datasets
!ls

/home/martin/PycharmProjects/no_joke/Short-Text-Corpus-For-Humor-Detection/datasets
humorous_oneliners.pickle      proverbs.pickle		 wiki_sentences.pickle
oneliners_incl_doubles.pickle  reuters_headlines.pickle


In [14]:
humorous_oneliners = pd.read_pickle('humorous_oneliners.pickle')
proverbs = pd.read_pickle('proverbs.pickle')
wiki_sentences = pd.read_pickle('wiki_sentences.pickle')
reuters_headlines = pd.read_pickle('reuters_headlines.pickle')

In [15]:
joke_sentences = humorous_oneliners

no_joke_sentences = proverbs + wiki_sentences + reuters_headlines
np.random.shuffle(no_joke_sentences)
no_joke_sentences = no_joke_sentences[:len(humorous_oneliners)]

In [16]:
sentences = []
labels = []

for joke in joke_sentences:
    sentences.append(joke)
    labels.append(1)
    
for no_joke in no_joke_sentences: 
    sentences.append(no_joke)
    labels.append(0)

In [17]:
print(f"Number of samples: {len(sentences)}")

Number of samples: 10502


In [18]:
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(sentences)

sequences = tokenizer.texts_to_sequences(sentences)
word_index = tokenizer.word_index

In [ ]:
tokenized_sentences = pad_sequences(sequences, maxlen=MAX_LEN)
labels = np.array(labels)

In [ ]:
# Shuffle the data
indices = np.arange(tokenized_sentences.shape[0])
np.random.shuffle(indices)

tokenized_sentences = tokenized_sentences[indices]
labels = labels[indices]

In [35]:
# Shuffle the data
indices = np.arange(tokenized_sentences.shape[0])
np.random.shuffle(indices)

tokenized_sentences = tokenized_sentences[indices]
labels = labels[indices]